Importing and Preview

In [2]:
import pandas as pd
import os
data_dir = r"C:\Users\harsh\OneDrive\Desktop\Job\Temp resume\Arch\Data\01jun2025-31aug2025_form13f\01JUN2025-31AUG2025_form13f"
try:
    submission_df = pd.read_csv(os.path.join(data_dir, 'submission.tsv'), sep='\t', encoding='utf-8')
    coverpage_df = pd.read_csv(os.path.join(data_dir, 'coverpage.tsv'), sep='\t', encoding='utf-8')
    infotable_df = pd.read_csv(os.path.join(data_dir, 'infotable.tsv'), sep='\t', encoding='utf-8')
    
    print("Loaded dataframes successfully:")
    print("SUBMISSION preview:")
    print(submission_df.head())
    print("\nCOVERPAGE preview:")
    print(coverpage_df.head())
    print("\nINFOTABLE preview:")
    print(infotable_df.head())
except FileNotFoundError as e:
    print(f"File not found. Please check paths and filenames. Details: {e}")
except Exception as e:
    print(f"An error occurred while loading files: {e}")

C:\Users\harsh\AppData\Local\Temp\ipykernel_14004\548073302.py:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  infotable_df = pd.read_csv(os.path.join(data_dir, 'infotable.tsv'), sep='\t', encoding='utf-8')


Loaded dataframes successfully:
SUBMISSION preview:
       ACCESSION_NUMBER  FILING_DATE SUBMISSIONTYPE      CIK PERIODOFREPORT
0  0001569785-25-000006  31-JUL-2025         13F-NT  1569785    30-JUN-2025
1  0001963669-25-000003  31-JUL-2025         13F-HR  1963669    30-JUN-2025
2  0000844150-25-000011  31-JUL-2025         13F-NT  1700459    30-JUN-2025
3  0002011668-25-000003  31-JUL-2025         13F-HR  2011668    30-JUN-2025
4  0001114739-25-000004  31-JUL-2025         13F-HR  1114739    30-JUN-2025

COVERPAGE preview:
       ACCESSION_NUMBER REPORTCALENDARORQUARTER ISAMENDMENT  AMENDMENTNO  \
0  0002010029-25-000002             30-JUN-2025         NaN          NaN   
1  0001068804-25-000006             30-JUN-2025         NaN          NaN   
2  0000891943-25-000009             30-JUN-2025         NaN          NaN   
3  0000045403-25-000008             30-JUN-2025         NaN          NaN   
4  0002079571-25-000006             31-MAR-2024         NaN          NaN   

  AMENDMENTTYPE

Merging

In [4]:
master_df = pd.merge(submission_df, coverpage_df, on='ACCESSION_NUMBER', how='left', suffixes=('_submission', '_coverpage'))
print("\nJoined SUBMISSION and COVERPAGE. Here's a preview:")
print(master_df.head())

master_df = pd.merge(master_df, infotable_df, on='ACCESSION_NUMBER', how='left')
print("\nAdded INFOTABLE data. Here's the updated preview:")
print(master_df.head())

print(f"\nFinal dataset shape: {master_df.shape}")

master_df.to_csv(os.path.join(data_dir, 'master_data.tsv'), sep='\t', index=False)
print("\nMaster DataFrame saved as 'master_data.tsv'.")


Joined SUBMISSION and COVERPAGE. Here's a preview:
       ACCESSION_NUMBER  FILING_DATE SUBMISSIONTYPE      CIK PERIODOFREPORT  \
0  0001569785-25-000006  31-JUL-2025         13F-NT  1569785    30-JUN-2025   
1  0001963669-25-000003  31-JUL-2025         13F-HR  1963669    30-JUN-2025   
2  0000844150-25-000011  31-JUL-2025         13F-NT  1700459    30-JUN-2025   
3  0002011668-25-000003  31-JUL-2025         13F-HR  2011668    30-JUN-2025   
4  0001114739-25-000004  31-JUL-2025         13F-HR  1114739    30-JUN-2025   

  REPORTCALENDARORQUARTER ISAMENDMENT  AMENDMENTNO AMENDMENTTYPE  \
0             30-JUN-2025         NaN          NaN           NaN   
1             30-JUN-2025         NaN          NaN           NaN   
2             30-JUN-2025         NaN          NaN           NaN   
3             30-JUN-2025         NaN          NaN           NaN   
4             30-JUN-2025         NaN          NaN           NaN   

  CONFDENIEDEXPIRED  ...                FILINGMANAGER_STREET2  \

Messiness Assessment

In [6]:
try:
    data_dir = r"C:\Users\harsh\OneDrive\Desktop\Job\Temp resume\Arch\Data\01jun2025-31aug2025_form13f\01JUN2025-31AUG2025_form13f"
    master_df = pd.read_csv(os.path.join(data_dir, 'master_data.tsv'), sep='\t', encoding='utf-8')
    print("Master DataFrame loaded successfully for assessment.")
except FileNotFoundError:
    print("Error: 'master_data.tsv' not found. Please run the previous notebook first.")
    exit()

print("\nStarting data quality assessment...")

missing_values = master_df.isnull().sum()
missing_percent = (missing_values / len(master_df)) * 100
missing_df = pd.DataFrame({'total_missing': missing_values, 'percent_missing': missing_percent})
missing_df = missing_df[missing_df['total_missing'] > 0].sort_values(by='percent_missing', ascending=False)

print("\nMissing values by column:")
print(missing_df)

print("\nData types for each column:")
print(master_df.info())

print("\nChecking variety in 'PERIODOFREPORT' values:")
print(master_df['PERIODOFREPORT'].value_counts(dropna=False).head(10))

print("\nSampling 'VALUE' column to compare pre- and post-2023 filings:")

master_df['FILING_DATE_DT'] = pd.to_datetime(master_df['FILING_DATE'], format='%d-%b-%Y', errors='coerce')

pre_2023_values = master_df[master_df['FILING_DATE_DT'].dt.year < 2023]['VALUE'].dropna().head(5)
post_2023_values = master_df[master_df['FILING_DATE_DT'].dt.year >= 2023]['VALUE'].dropna().head(5)

print("\nSample 'VALUE' entries from before 2023 (expected in thousands):")
print(pre_2023_values)

print("\nSample 'VALUE' entries from 2023 and later (expected in dollars):")
print(post_2023_values)

initial_rows = master_df.shape[0]
master_df = master_df.drop_duplicates(subset=['ACCESSION_NUMBER', 'INFOTABLE_SK'], keep='first')
duplicate_count = initial_rows - master_df.shape[0]

print(f"\nRemoved {duplicate_count} duplicate rows based on ACCESSION_NUMBER and INFOTABLE_SK.")

C:\Users\harsh\AppData\Local\Temp\ipykernel_14004\1672579817.py:3: DtypeWarning: Columns (6,8,9,10,11,12,15,18,22,24,29,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(os.path.join(data_dir, 'master_data.tsv'), sep='\t', encoding='utf-8')


Master DataFrame loaded successfully for assessment.

Starting data quality assessment...

Missing values by column:
                              total_missing  percent_missing
DATEDENIEDEXPIRED                   3362012        99.991613
DATEREPORTED                        3362012        99.991613
REASONFORNONCONFIDENTIALITY         3362012        99.991613
CONFDENIEDEXPIRED                   3271131        97.288667
PUTCALL                             3231410        96.107301
AMENDMENTTYPE                       3149421        93.668817
AMENDMENTNO                         3149421        93.668817
ADDITIONALINFORMATION               3116394        92.686541
FIGI                                2992700        89.007683
CRDNUMBER                           2245004        66.770009
SECFILENUMBER                       2238787        66.585105
ISAMENDMENT                         1959916        58.291036
OTHERMANAGER                        1855065        55.172599
FILINGMANAGER_STREET2        

Cleaning Pipeline

In [8]:
# 3. Data Cleaning Pipeline (The 'Keystone' Treatment Plan)
# This is where we create a function to clean the data. This shows your
# ability to build a repeatable, automated process.
def clean_form13f_data(df):
    """
    Cleans and standardizes the master Form 13F DataFrame.

    This function performs several key data quality improvements:
    1. Standardizes date formats to YYYY-MM-DD.
    2. Corrects the 'VALUE' column inconsistency based on filing date.
    3. Fills missing categorical values with 'Unspecified' or a similar placeholder.
    4. Converts relevant columns to appropriate data types.
    """
    cleaned_df = df.copy()

    # Treatment 1: Standardize Dates
    print("  -> Applying Treatment: Standardizing date formats...")
    # Convert 'FILING_DATE' and 'PERIODOFREPORT' to a uniform datetime format
    cleaned_df['FILING_DATE'] = pd.to_datetime(cleaned_df['FILING_DATE'], format='%d-%b-%Y', errors='coerce').dt.strftime('%Y-%m-%d')
    cleaned_df['PERIODOFREPORT'] = pd.to_datetime(cleaned_df['PERIODOFREPORT'], format='%d-%b-%Y', errors='coerce').dt.strftime('%Y-%m-%d')

    # Treatment 2: Correcting the VALUE field inconsistency
    print("  -> Applying Treatment: Correcting 'VALUE' reporting inconsistency...")
    # The README indicates values before 2023 were in thousands.
    # We'll use a hardcoded date to represent this historical change for demonstration purposes.
    cleaned_df['FILING_DATE_DT'] = pd.to_datetime(cleaned_df['FILING_DATE'])
    # Identify rows that need a value correction
    pre_2023_mask = cleaned_df['FILING_DATE_DT'] < '2023-01-01'
    # Multiply these values by 1000 to normalize them to dollars.
    cleaned_df.loc[pre_2023_mask, 'VALUE'] = cleaned_df.loc[pre_2023_mask, 'VALUE'] * 1000

    # Treatment 3: Handling Missing Values (the "Gaps")
    print("  -> Applying Treatment: Filling in missing values...")
    # Fill categorical NaN values with 'Unspecified'
    # The column 'ISCONFIDENTIALOMITTED' from the original code was not present in the merged dataframe.
    # We are using columns that are confirmed to be in the dataframe from our initial analysis.
    categorical_cols = ['AMENDMENTTYPE', 'REASONFORNONCONFIDENTIALITY', 'PUTCALL', 'FIGI', 'FILINGMANAGER_STATEORCOUNTRY', 'CONFDENIEDEXPIRED']
    for col in categorical_cols:
        cleaned_df[col] = cleaned_df[col].fillna('Unspecified')
    
    # Fill numeric NaNs with 0
    # Columns like TABLEENTRYTOTAL, TABLEVALUETOTAL, and OTHERINCLUDEDMANAGERSCOUNT
    # were not found in the merged data from our initial analysis, so they are removed.
    numeric_cols_to_fill = ['AMENDMENTNO', 'CRDNUMBER', 'SECFILENUMBER', 'VOTING_AUTH_SOLE', 'VOTING_AUTH_SHARED', 'VOTING_AUTH_NONE']
    for col in numeric_cols_to_fill:
        cleaned_df[col] = cleaned_df[col].fillna(0)

    # Treatment 4: Data Type Conversion
    print("  -> Applying Treatment: Converting data types...")
    # Convert `VALUE` to integer after filling NaN
    cleaned_df['VALUE'] = pd.to_numeric(cleaned_df['VALUE'], errors='coerce').fillna(0).astype('int64')

    print("Data cleaning complete.")
    return cleaned_df

# Apply the cleaning pipeline to our master DataFrame
master_df = clean_form13f_data(master_df)

print("\n--- Data Quality Assessment: Final State (Post-Cleaning) ---")

# Re-assess missing values to confirm the treatment worked
missing_values_post = master_df.isnull().sum()
missing_percent_post = (missing_values_post / len(master_df)) * 100
missing_df_post = pd.DataFrame({'total_missing': missing_values_post, 'percent_missing': missing_percent_post})
print("\n- Missing Values Per Column (Post-Cleaning):")
print(missing_df_post[missing_df_post['total_missing'] > 0])

# Re-assess date formats
print("\n- Sample of 'PERIODOFREPORT' column for consistency check (Post-Cleaning):")
print(master_df['PERIODOFREPORT'].value_counts(dropna=False).head(5))

# Re-assess value normalization
print("\n- Sample of 'VALUE' column (Post-Cleaning):")
print(master_df['VALUE'].head(10))

# This section of the notebook is now complete. We have a clean dataset ready for the next step.
# It is also a powerful proof-of-concept for your slide deck.
master_df.to_csv(os.path.join(data_dir, 'cleaned_master_data.tsv'), sep='\t', index=False)
print("\nCleaned Master DataFrame saved as 'cleaned_master_data.tsv'.")


  -> Applying Treatment: Standardizing date formats...
  -> Applying Treatment: Correcting 'VALUE' reporting inconsistency...
  -> Applying Treatment: Filling in missing values...
  -> Applying Treatment: Converting data types...
Data cleaning complete.

--- Data Quality Assessment: Final State (Post-Cleaning) ---

- Missing Values Per Column (Post-Cleaning):
                       total_missing  percent_missing
ISAMENDMENT                  1959916        58.291036
DATEDENIEDEXPIRED            3362012        99.991613
DATEREPORTED                 3362012        99.991613
FILINGMANAGER_STREET1             22         0.000654
FILINGMANAGER_STREET2        1630007        48.479015
FILINGMANAGER_CITY                50         0.001487
FILINGMANAGER_ZIPCODE            885         0.026321
ADDITIONALINFORMATION        3116394        92.686541
INFOTABLE_SK                    1988         0.059126
NAMEOFISSUER                    2157         0.064153
TITLEOFCLASS                    2492        